In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import pandas as pd
import os
import random
import numpy as np
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

random.seed(100)

lst_file = ['period1.csv', 'period2.csv', 'period3.csv', 'period4.csv']

In [3]:
def load_data(filename):
    headers = ['Large_BP', 'Large_ROE', 'Large_SP',
       'Large_Return_Rate_last_quarter', 'Large_Market_Value',
       'Small_Systematic_Risk', 'Annual_Return', 'Excess_Return',
       'Systematic_Risk', 'Total_Risk', 'Abs_Win_Rate', 'Rel_Win_Rate']

    train = pd.read_csv(filename, index_col = 0)
    train.columns = headers

    Xtrain = train.iloc[:,:6]
    ytrain = train.iloc[:,6:13]
    #ytrain = np.array(ytrain).ravel()
    return [Xtrain,ytrain]


In [4]:
lst_data = []
for i in range(len(lst_file)):
    data = load_data(filename=lst_file[i])
    lst_data.append(data)

In [5]:
lst_data[0][1].head(5)

,Annual_Return,Excess_Return,Systematic_Risk,Total_Risk,Abs_Win_Rate,Rel_Win_Rate
ID,,,,,,
1,0.625,0.714,0.382,0.688,0.56,0.500
2,0.599,0.431,0.737,0.634,0.68,0.725
3,0.518,0.590,0.443,0.777,0.56,0.200
4,0.396,0.320,0.682,0.788,0.32,0.500
5,0.219,0.351,0.381,0.200,0.56,0.275


In [6]:
def calculate_coef(data_train, data_test):
    Xtrain = data_train[0]
    ytrain = data_train[1]
    Xtest  = data_test[0]
    ytest  = data_test[1]
    # choose your number layer and number neurons
    mlp = MLPRegressor(hidden_layer_sizes=(10,30,10), max_iter=6000, random_state=0)
    mlp.fit(Xtrain,ytrain)

    # Pridicts 
    predictions = mlp.predict(Xtest)
    #print(predictions)
    #targets = Y_test.values
    #score = mlp.score(Xtest, ytest)
    lst_corelation = []
    for i in range(0,6):
        corelation = np.corrcoef(predictions[:,i], ytest[ytest.columns[i]])
        lst_corelation.append(str(round(corelation[0,1],3)))
    #return corelation
    
    lst_mse = []
    for i in range(0,6):
        mse = mean_squared_error(ytest[ytest.columns[i]], predictions[:,i])  
        lst_mse.append(str(round(mse,3)))
    
    return lst_corelation,lst_mse

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [7]:

def test_error(data_train, data_test):  
    Xtrain = data_train[0]
    ytrain = data_train[1]
    Xtest  = data_test[0]
    ytest  = data_test[1]
    # choose your number layer and number neurons
    mlp = MLPRegressor(hidden_layer_sizes=(10,30,10), max_iter=6000, random_state=0)
    mlp.fit(Xtrain,ytrain)
    print("mlp.coefs_: ", len(mlp.coefs_))
    #weight_matrix = mlp.coefs_[1].dot(mlp.coefs_[2])
    #tb = pd.DataFrame()
    predictions = mlp.predict(Xtest)
    #print(predictions.shape)
    #print(ytest[ytest.columns[0]])
    lst_corelation = []
    for i in range(0,6):
        corelation = np.corrcoef(predictions[:,i], ytest[ytest.columns[i]])
        lst_corelation.append(str(round(corelation[0,1],3)))
    #print("Core: ", lst_corelation)
    #score = mlp.score(Xtest, ytest)
    score = mean_squared_error(ytest, predictions)
    print("Squared score: ",score)
    print("Absolute: ", mean_absolute_error(ytest, predictions))
    print("Mean Absolute Percentage Error: ", mean_absolute_percentage_error(ytest, predictions))
    print("Mean y_test: ",np.mean(ytest))
    #graph = seaborn.lmplot('$R$', 'Equilibrium Value', data = df, fit_reg=True, order=2, ci=None)
    #graph.set(xlim = (-0.25,10))
test_error(data_train=lst_data[0], data_test=lst_data[1])

mlp.coefs_:  4
Squared score:  0.0216231190065
Absolute:  0.120100666284
Mean Absolute Percentage Error:  31.6914653115
Mean y_test:  Annual_Return      0.473905
Excess_Return      0.550159
Systematic_Risk    0.430397
Total_Risk         0.352286
Abs_Win_Rate       0.442095
Rel_Win_Rate       0.482540
dtype: float64


In [8]:
def calculate_tables(data_train):
    Xtrain = data_train[0]
    ytrain = data_train[1]
    # choose your number layer and number neurons
    mlp = MLPRegressor(hidden_layer_sizes=(10, 30, 10), max_iter=6000, random_state=0)
    mlp.fit(Xtrain,ytrain)
    weight_matrix = mlp.coefs_[0]
    for i in range(1, len(mlp.coefs_)):
        #print("mlp.coefs_: ",i)
        weight_matrix = weight_matrix.dot(mlp.coefs_[i])
    tb = pd.DataFrame()
    headers = ['Large_BP', 'Large_ROE', 'Large_SP','LRR_last_quarter', 'Large_Market_Value',
           'Small_Systematic_Risk']
    for i in range(len(headers)):
        tb[headers[i]] = weight_matrix[i]
    return tb


In [9]:
#test_w = calculate_weight(feature=mlp.coefs_[1], outcome=mlp.coefs_[2])

In [10]:
df_corr = pd.DataFrame({ 'Period' : ['Annual_Return', 'Excess_Return',
       'Systematic_Risk', 'Total_Risk', 'Abs_Win_Rate', 'Rel_Win_Rate']})
df_mse = pd.DataFrame({ 'Period' : ['Annual_Return', 'Excess_Return',
       'Systematic_Risk', 'Total_Risk', 'Abs_Win_Rate', 'Rel_Win_Rate']})
for i in range(len(lst_data)-1):
    for j in range(i+1, len(lst_data)):
        #print(i,j)
        coef,mse = calculate_coef(lst_data[i], lst_data[j])
        #print("Coef: ", coef)
        #train_data = lst_data[i]
        title = "Train"+ str(i+1) + '/' + 'Test'+ str(j+1)
        #lst_annual.append(coef)
        df_corr[title] = coef
        df_mse[title] = mse
df_mse = df_mse.set_index('Period')
df_mse.loc['Average'] = round(df_mse.astype(float).mean(),3)
lst_ave_mse = []
for i in range(df_mse.shape[0]):
    lst_ave_mse.append(str(round(np.mean(df_mse.iloc[i:i+1,1:].astype(float).values.tolist()),4)))
df_mse['Average'] = lst_ave_mse

writer = pd.ExcelWriter('Correlation_coefficients.xlsx')
df_corr.to_excel(writer,'Correlation_coefficients')
writer = pd.ExcelWriter('MeanSquareError.xlsx')
df_mse.to_excel(writer,'MeanSquareError')

In [11]:
#Correlation coefficients tables
df_corr

,Period,Train1/Test2,Train1/Test3,Train1/Test4,Train2/Test3,Train2/Test4,Train3/Test4
0,Annual_Return,-0.119,0.751,0.42,-0.385,0.113,0.572
1,Excess_Return,0.009,0.76,0.236,-0.516,-0.077,0.467
2,Systematic_Risk,-0.173,0.154,0.078,-0.259,-0.53,0.239
3,Total_Risk,0.346,0.512,0.055,0.594,-0.112,-0.158
4,Abs_Win_Rate,0.262,0.495,0.207,0.048,0.4,0.047
5,Rel_Win_Rate,0.28,0.421,0.144,-0.028,0.283,0.021


In [12]:
#Mean square error tables
df_mse

,Train1/Test2,Train1/Test3,Train1/Test4,Train2/Test3,Train2/Test4,Train3/Test4,Average
Period,,,,,,,
Annual_Return,0.031,0.016,0.012,0.05,0.025,0.01,0.0226
Excess_Return,0.024,0.01,0.018,0.042,0.021,0.014,0.021
Systematic_Risk,0.024,0.023,0.025,0.029,0.034,0.02,0.0262
Total_Risk,0.02,0.015,0.023,0.04,0.031,0.027,0.0272
Abs_Win_Rate,0.015,0.019,0.025,0.044,0.058,0.028,0.0348
Rel_Win_Rate,0.016,0.029,0.02,0.046,0.02,0.034,0.0298
Average,0.022,0.019,0.02,0.042,0.032,0.022,0.027


In [13]:
#Table weight
headers = ['Large_BP', 'Large_ROE', 'Large_SP','LRR_last_quarter', 'Large_Market_Value',
           'Small_Systematic_Risk']

In [14]:
pd_Annual_Return   = pd.DataFrame(columns=headers)
pd_Excess_Return   = pd.DataFrame(columns=headers)
pd_Systematic_Risk = pd.DataFrame(columns=headers)
pd_Total_Risk      = pd.DataFrame(columns=headers)
pd_Abs_Win_Rate    = pd.DataFrame(columns=headers)
pd_Rel_Win_Rate    = pd.DataFrame(columns=headers)
for i in range(len(lst_data)):
    #print(i,j)
    period = calculate_tables(data_train=lst_data[i])
    #break
    #print(period)
    pd_Annual_Return = pd_Annual_Return.append(period.loc[0:0,:], ignore_index=True)
    pd_Excess_Return = pd_Excess_Return.append(period.loc[1:1,:], ignore_index=True)
    pd_Systematic_Risk = pd_Systematic_Risk.append(period.loc[2:2,:], ignore_index=True)
    pd_Total_Risk = pd_Total_Risk.append(period.loc[3:3,:], ignore_index=True)
    pd_Abs_Win_Rate = pd_Abs_Win_Rate.append(period.loc[4:4,:], ignore_index=True)
    pd_Rel_Win_Rate = pd_Rel_Win_Rate.append(period.loc[5:5,:], ignore_index=True)

In [15]:
def calculate_ave(target):
    target.loc['Average'] = round(target.astype(float).mean(),4)
    lst_ave = []
    for i in range(target.shape[0]):
        lst_ave.append(str(round(np.mean(target.iloc[i:i+1,1:].astype(float).values.tolist()),4)))
    target['Average'] = lst_ave
    return target

In [16]:
#Effects of weights
pd_Annual_Return = calculate_ave(pd_Annual_Return)
pd_Annual_Return

,Large_BP,Large_ROE,Large_SP,LRR_last_quarter,Large_Market_Value,Small_Systematic_Risk,Average
0,0.302787,0.483660,0.300627,-0.083643,0.051088,0.065687,0.1635
1,-0.020551,0.204737,-0.222644,0.074985,0.134191,0.077261,0.0537
2,0.279807,0.308764,0.312765,-0.102041,0.081623,0.190086,0.1582
3,0.181519,0.249305,0.166358,0.064583,0.104362,0.068555,0.1306
Average,0.185900,0.311600,0.139300,-0.011500,0.092800,0.100400,0.1265


In [18]:
import matplotlib.pyplot as plt

def visualize_weight(outcome_name):
    pdMeans = []
    pdStd = []
    ind = np.arange(len(outcome_name.columns))
    for head in outcome_name.columns:
        #print(head)
        pdMeans.append(np.mean(outcome_name[head][:-1]))
        pdStd.append(np.std(outcome_name[head][:-1]))

    width = 0.35       # the width of the bars: can also be len(x) sequence

    fig = plt.figure(figsize=(12, 5))
    p1 = plt.bar(ind, pdMeans, width, yerr=pdStd)

    plt.ylabel('Scores')
    plt.title('Scores by Annual_Return')
    plt.xticks(ind, outcome_name.columns)
    plt.yticks(np.arange(0, 1, 0.2))
    #plt.legend(p1[0], 'Annual_Return')

    plt.show()
    
visualize_weight(outcome_name=pd_Annual_Return)   

TypeError: Could not convert 0.16350.05370.15820.1306 to numeric

In [ ]:
#Effects of weights
pd_Excess_Return = calculate_ave(pd_Excess_Return)
pd_Excess_Return 

In [ ]:
#Effects of weights
pd_Systematic_Risk = calculate_ave(pd_Systematic_Risk)
pd_Systematic_Risk 

In [ ]:
#Effects of weights
pd_Total_Risk = calculate_ave(pd_Total_Risk)
pd_Total_Risk

In [ ]:
#Effects of weights
pd_Abs_Win_Rate = calculate_ave(pd_Abs_Win_Rate)
pd_Abs_Win_Rate

In [ ]:
#Effects of weights
pd_Rel_Win_Rate = calculate_ave(pd_Rel_Win_Rate)
pd_Rel_Win_Rate

In [ ]:
writer = pd.ExcelWriter('Effects_of_weights.xlsx')
pd_Annual_Return.to_excel(writer,'Annual_Return')
pd_Excess_Return.to_excel(writer,'Excess_Return')
pd_Systematic_Risk.to_excel(writer,'Systematic_Risk')
pd_Total_Risk.to_excel(writer,'Total_Risk')
pd_Abs_Win_Rate.to_excel(writer,'Abs_Win_Rate')
pd_Rel_Win_Rate.to_excel(writer,'Rel_Win_Rate')